# Computing the distance to the ideal solution
Given a Pareto-optimal frontier of solutions to a multi-criterion optimization model, this program computes the average distance from frontier points to the model's ideal solution. This provides a measure of the conflict among the objectives.

In [24]:
import pandas as pd
import numpy as np

### Preparation for volume computation algorithm

In [45]:
# Get the solutions that define the pareto frontier.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

sols = pd.read_csv("../solutionSets/3d/ClimateChange_E85/climateChange_EfficientSolutions_E85.csv")

In [46]:
# Hard-coded manipulations to tidy-up sols

sols = sols.drop(["SolutionIndex","Frontier"],axis=1) #remove solution index and frontier columns (climate change sets)
#sols = sols.drop(["SolutionIndex"],axis=1) #remove solution index column (pack forest and Chilean sets)
# nothing for some sets (toth mcDill, small sed fire)

In [47]:
# Get list of objectives from the dataframe

objs = sols.columns.tolist()

In [48]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the dataframe

objSenses = {}

# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 0
objSenses["Northern Spotted Owl Habitat (ha)"] = 1
#objSenses["PER"] = 0
#objSenses

In [49]:
def maxAll(origf,objSenses):
    
    f = origf
    
    # get bounds on the objectives' values
    objBounds = {}
    for obj,sense in objSenses.items():
        objBounds[obj] = [f[obj].min(),f[obj].max()]
    # get each objective's ideal value
    idealObjVals = {}
    for obj,bounds in objBounds.items():
        idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max
    # get each objective's worst value
    worstObjVals = {}
    for obj,bounds in objBounds.items():
        worstObjVals[obj] = bounds[not(objSenses[obj])]
        
    # Convert each objective to a maximization

    for obj,bounds in objBounds.items():
        f[obj] = f[obj].apply(
            lambda x: \
            abs(x - worstObjVals[obj]))
    
    return f

In [50]:
def maxAllAndNorm(origf,objSenses):
    
    f = origf
    
    # get bounds on the objectives' values
    objBounds = {}
    for obj,sense in objSenses.items():
        objBounds[obj] = [f[obj].min(),f[obj].max()]
    # get each objective's ideal value
    idealObjVals = {}
    for obj,bounds in objBounds.items():
        idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max
    # get each objective's worst value
    worstObjVals = {}
    for obj,bounds in objBounds.items():
        worstObjVals[obj] = bounds[not(objSenses[obj])]
        
    # Normalize the objective spaces by converting each value to the relative achievement along its axis:

    #   distance from the worst case value
    # ---------------------------------------
    # total distance spanned by the objective

    # Normalized objective space is in [0,1]^N where N = # of objectives


    for obj,bounds in objBounds.items():
        f[obj] = f[obj].apply(
            lambda x: \
            abs(x - worstObjVals[obj]) / \
            (bounds[1]-bounds[0]))
    
    return f

### Computing the frontier's average distance to ideal solution

In [51]:
# Convert and normalize the data
# Define the ideal solution

normalize = False

if normalize:
    sols = maxAllAndNorm(sols,objSenses)
    idealSolution = np.ones(len(objs))
else:
    sols = maxAll(sols,objSenses)
    idealSolution = np.asarray([max(sols[obj]) for obj in objs])

In [52]:
# Initialize average distance to ideal

frontierAvgDist= 0

In [53]:
# Compute average distance

for solution in sols.itertuples(index=False,name=None):
    frontierAvgDist += np.linalg.norm(solution - idealSolution)
    
frontierAvgDist /= len(sols)

In [54]:
frontierAvgDist

337.86821059276934